# How to recursively split text by characters
This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ```["\n\n", "\n", " ", ""]```. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

1. How the text is split: by list of characters.
2. How the chunk size is measured: by number of characters.
Below we show example usage.

To obtain the string content directly, use ```.split_text```.

To create LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) objects (e.g., for use in downstream tasks), use ```.create_documents```.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Splitting straight from file

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])
print(texts[1])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and'
page_content='of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.'


In [10]:
texts = text_splitter.create_documents([state_of_the_union])

In [12]:
len(texts)

570

In [20]:
texts[0]

Document(metadata={}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and')

# Loading the file and then splitting it

In [15]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader("./state_of_the_union.txt")


In [16]:
docs = []
for doc in loader.lazy_load():
    docs.append(doc)

INFO: NumExpr defaulting to 4 threads.


In [18]:
len(docs)

359

In [19]:
docs[0]

Document(metadata={'source': './state_of_the_union.txt', 'last_modified': '2024-10-26T17:11:01', 'languages': ['eng'], 'file_directory': '.', 'filename': 'state_of_the_union.txt', 'filetype': 'text/plain', 'category': 'NarrativeText', 'element_id': 'f63ede2dc1fdce051d121c62fc11e725'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.')

In [21]:
split_docs = text_splitter.split_documents(docs)


In [22]:
len(split_docs)

580

In [23]:
split_docs[0]

Document(metadata={'source': './state_of_the_union.txt', 'last_modified': '2024-10-26T17:11:01', 'languages': ['eng'], 'file_directory': '.', 'filename': 'state_of_the_union.txt', 'filetype': 'text/plain', 'category': 'NarrativeText', 'element_id': 'f63ede2dc1fdce051d121c62fc11e725'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and')

# Splitting text from languages without word boundaries

Some writing systems do not have word boundaries, for example Chinese, Japanese, and Thai. Splitting text with the default separator list of ```["\n\n", "\n", " ", ""]``` can cause words to be split between chunks. To keep words together, you can override the list of separators to include additional punctuation:

- Add ASCII full-stop "```.```", [Unicode fullwidth](https://en.wikipedia.org/wiki/Halfwidth_and_Fullwidth_Forms_(Unicode_block)) full stop "```．```" (used in Chinese text), and [ideographic full stop](https://en.wikipedia.org/wiki/CJK_Symbols_and_Punctuation) "```。```" (used in Japanese and Chinese)
- Add [Zero-width](https://en.wikipedia.org/wiki/Zero-width_space) space used in Thai, Myanmar, Kmer, and Japanese.
- Add ASCII comma "```,```", Unicode fullwidth comma "```，```", and Unicode ideographic comma "```、```"

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    # Existing args
)

# Split with language (for code)

In [ ]:
from langchain_text_splitters import Language

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(docs)
len(texts)